In [1]:
import pandas as pd
import numpy as np
import glob

#### Data import and cleaning

Data Source: https://outbreak.sgcharts.com/data

In [2]:
cases_df = pd.DataFrame()
shape = 0
for file in glob.glob('../Data/sgcharts/csv/*'):
    df = pd.read_csv(file,header = None)
    shape += df.shape[0]
    cases_df = cases_df.append(df)

print(shape)
cases_df.head()

56976


,0,1,2,3,4,5,6,7,8
0,4,bishan street 22 (block 232),1.358286,103.845226,1,24,83,150703,7
1,7,bishan street 22 (block 233),1.358639,103.845259,1,24,83,150703,7
2,3,bishan street 22 (block 234),1.358390,103.845955,1,24,83,150703,7
3,1,bishan street 22 (block 235),1.358719,103.846477,1,24,83,150703,7
4,4,bishan street 22 (block 236),1.359041,103.846849,1,24,83,150703,7


In [3]:
# cases_df2 = pd.DataFrame()
# shape = 0
# for file in glob.glob('../Data/sgcharts/incorrect_latitude_longitude/*.csv'):
#     df = pd.read_csv(file,header = None)
#     shape += df.shape[0]
#     cases_df2 = cases_df2.append(df)

# print(shape)
# cases_df2.head()

In [4]:
cases_df.shape

(56976, 9)

In [5]:
cases_df.columns = ['no_of_cases','address','latitude','longitude','cluster_no','recent_cases','total_cases','date','month']

In [6]:
cases_df.reset_index(drop = True, inplace = True)

In [7]:
cases_df.tail()

,no_of_cases,address,latitude,longitude,cluster_no,recent_cases,total_cases,date,month
56971,1,tampines street 71 (block 715),1.359660,103.937074,131,2,2,201106,11
56972,1,tampines street 71 (block 716),1.359482,103.936586,131,2,2,201106,11
56973,1,tampines street 82 (block 842b),1.351145,103.936991,132,2,2,201106,11
56974,1,tampines street 82 (block 842e),1.351754,103.938303,132,2,2,201106,11
56975,2,teo kim eng road,1.328131,103.908562,133,2,2,201106,11


In [8]:
cases_df['date'] = pd.to_datetime(cases_df['date'], format='%y%m%d')

In [9]:
cases_df['recent_cases'].value_counts()

2     12617
3      8018
1      6675
4      5716
5      3927
6      2946
7      2377
8      2140
9      1806
10     1486
11     1250
12     1044
13      844
14      590
17      519
16      488
15      426
24      389
21      352
18      345
22      316
20      291
19      274
25      221
26      206
28      187
35      178
33      156
27      135
39      114
30      109
34      100
31       97
38       84
29       78
40       77
37       77
23       73
32       48
42       39
92       35
61       33
47       30
55       29
43       21
36        7
41        6
Name: recent_cases, dtype: int64

In [10]:
# pip install geopy

In [11]:
# Remove duplicates so that the reverse geocoding runs faster
coords_to_map = cases_df.drop_duplicates(['address'])[['address','latitude','longitude']]
coords_to_map.shape

(9777, 3)

In [147]:
import geopy

def get_zipcode(df, geolocator, lat_field, lon_field):
    try:
        location = geolocator.reverse((df[lat_field], df[lon_field]))
        return location.raw['address']['postcode']
    except: 
        return 0

def get_suburb(df, geolocator, lat_field, lon_field):
    try:
        location = geolocator.reverse((df[lat_field], df[lon_field]))
        return location.raw['address']['suburb']
    except: 
        return "-"


geolocator = geopy.Nominatim(user_agent='1234')

coords_to_map['postal_code'] = coords_to_map.apply(get_zipcode, axis=1, geolocator=geolocator, lat_field='latitude', lon_field='longitude')
coords_to_map['suburb'] = coords_to_map.apply(get_suburb, axis=1, geolocator=geolocator, lat_field='latitude', lon_field='longitude')

In [152]:
coords_to_map

,address,latitude,longitude,postal_code,suburb
0,bishan street 22 (block 232),1.358286,103.845226,570282,Bishan
1,bishan street 22 (block 233),1.358639,103.845259,570233,Bishan
2,bishan street 22 (block 234),1.358390,103.845955,570233,Bishan
3,bishan street 22 (block 235),1.358719,103.846477,570227,Bishan
4,bishan street 22 (block 236),1.359041,103.846849,570227,Bishan
...,...,...,...,...,...
56964,simei street 4 (block 234),1.343434,103.955341,520234,Tampines
56968,stirling road (block 169),1.290476,103.803114,140169,-
56969,tampines street 45 (block 492d),1.363751,103.953811,523492,Tampines
56970,tampines street 45 (block 494j),1.364231,103.954394,529494,Tampines


In [282]:
cases_df2 = cases_df.merge(coords_to_map[['address','postal_code','suburb']], how = 'left', on = 'address')

Map the remaining postal codes to their respective area if the reverse geocoder did not manage to return an area
([*source*](https://www.ura.gov.sg/realEstateIIWeb/resources/misc/list_of_postal_districts.htm))

In [288]:
mapping = pd.read_csv('../Data/postal_mapping.csv')

In [289]:
mapping.head()

,Postal District,Postal Sector,General Location
0,1,"01, 02, 03, 04, 05, 06","Raffles Place, Cecil, Marina, People's Park"
1,2,"07, 08","Anson, Tanjong Pagar"
2,3,"14,15, 16","Queenstown, Tiong Bahru"
3,4,"09, 10","Telok Blangah, Harbourfront"
4,5,"11, 12, 13","Pasir Panjang, Hong Leong Garden, Clementi New..."


In [283]:
def mapping_postal(row):
    if row['suburb'] == "-":
        postal = row['postal_code'][:2]
        return mapping.loc[mapping['Postal Sector'].str.contains(postal),'General Location'].reset_index()['General Location'][0]
    else:
        return row['suburb']

In [284]:
cases_df2['suburb'] = cases_df2.apply(mapping_postal,axis=1)

In [287]:
# Save a WIP ver just in case, as the reverse geocoding takes hours to run
cases_df2.to_csv('../Data/dengue_cases_region_wip.csv', index = False)

In [12]:
# Open the WIP ver to skip the above steps
cases_df2 = pd.read_csv('../Data/dengue_cases_region_wip.csv')

In [13]:
cases_region = cases_df2['suburb'].unique()
cases_region

array(['Bishan', 'Yishun', 'Geylang', 'Sembawang', 'Tampines', 'Simei',
       'Changi', 'Bedok', 'Jurong West', 'Ulu Pandan', 'Holland Village',
       'Jurong East', 'Serangoon', 'Woodlands', 'Bukit Timah',
       'Choa Chu Kang', 'Seng Kang', 'Hougang', 'Bukit Merah', 'Punggol',
       'Tanglin', 'Queenstown, Tiong Bahru', 'Pasir Ris', 'Ang Mo Kio',
       'Bukit Batok', 'Rochor', 'Toa Payoh', 'Marine Parade', 'Jurong',
       'Serangoon North', 'Clementi', 'Kallang', 'Joo Seng',
       'Pasir Panjang, Hong Leong Garden, Clementi New Town',
       'Bukit Panjang', 'Downtown Core', 'Novena', 'Serangoon Gardens',
       'Upper Bukit Timah, Clementi Park, Ulu Pandan', 'Hillview',
       'Tai Seng', 'Mandai', 'Changi Business Park', 'Kaki Bukit',
       'one-north', 'Kallang Way', 'Sungei Kadut', 'Singapore River',
       'Outram', 'Potong Pasir', 'Western Islands', 'Paya Lebar',
       'Newton', 'Pasir Panjang', 'Tuas', 'Western Water Catchment',
       'Central Water Catchment', 'Sele

In [14]:
# Read in the weather data's region to clean up the suburb
weather = pd.read_csv('../Data/weather.csv')
region_list = weather['region'].unique()
region_list

array(['Admiralty', 'Admiralty West', 'Ang Mo Kio', 'Boon Lay (East)',
       'Boon Lay (West)', 'Botanic Garden', 'Buangkok', 'Bukit Panjang',
       'Bukit Timah', 'Buona Vista', 'Chai Chee', 'Changi',
       'Choa Chu Kang (Central)', 'Choa Chu Kang (South)',
       'Choa Chu Kang (West)', 'Clementi', 'Dhoby Ghaut',
       'East Coast Parkway', 'Jurong (East)', 'Jurong (North)',
       'Jurong (West)', 'Jurong Island', 'Jurong Pier', 'Kampong Bahru',
       'Kent Ridge', 'Khatib', 'Kranji Reservoir', 'Lim Chu Kang',
       'Lower Peirce Reservoir', 'Macritchie Reservoir', 'Mandai',
       'Marina Barrage', 'Marine Parade', 'Newton', 'Nicoll Highway',
       'Pasir Panjang', 'Pasir Ris (Central)', 'Pasir Ris (West)',
       'Paya Lebar', 'Pulau Ubin', 'Punggol', 'Queenstown', 'Seletar',
       'Semakau Island', 'Sembawang', 'Sentosa Island', 'Serangoon',
       'Serangoon North', 'Simei', 'Somerset (Road)', 'Tai Seng',
       'Tanjong Katong', 'Tanjong Pagar', 'Tengah', 'Toa Payoh', 

In [15]:
print('These areas should be renamed: ')
for i in cases_region:
    if i not in region_list:     
        print(i)

These areas should be renamed: 
Bishan
Geylang
Tampines
Bedok
Jurong West
Holland Village
Jurong East
Woodlands
Choa Chu Kang
Seng Kang
Hougang
Bukit Merah
Tanglin
Queenstown, Tiong Bahru
Pasir Ris
Bukit Batok
Rochor
Jurong
Kallang
Joo Seng
Pasir Panjang, Hong Leong Garden, Clementi New Town
Downtown Core
Novena
Serangoon Gardens
Upper Bukit Timah, Clementi Park, Ulu Pandan
Hillview
Changi Business Park
Kaki Bukit
one-north
Kallang Way
Sungei Kadut
Singapore River
Outram
Potong Pasir
Western Islands
Western Water Catchment
Central Water Catchment
Katong
Pioneer
Orchard
River Valley
Alexandra
Museum


In [16]:
cases_df3 = cases_df2.replace({
    'Bishan': 'Ang Mo Kio',
    'Geylang': 'Tai Seng',
    'Tampines': 'Simei',
    'Bedok': 'Chai Chee',
    'Jurong West': 'Jurong (West)',
    'Holland Village': 'Buona Vista',
    'Jurong East': 'Jurong (East)',
    'Woodlands': 'Admiralty',
    'Choa Chu Kang': 'Choa Chu Kang (Central)',
    'Seng Kang': 'Buangkok',
    'Hougang': 'Buangkok',
    'Bukit Merah': 'Kampong Bahru',
    'Tanglin': 'Botanic Garden',
    'Queenstown, Tiong Bahru': 'Queenstown',
    'Pasir Ris': 'Pasir Ris (Central)',
    'Bukit Batok': 'Bukit Panjang',
    'Rochor': 'Dhoby Ghaut',
    'Jurong': 'Jurong (East)',
    'Kallang': 'Whampoa',
    'Joo Seng': 'Tai Seng',
    'Pasir Panjang, Hong Leong Garden, Clementi New Town': 'Pasir Panjang',
    'Downtown Core': 'Marina Barrage',
    'Novena': 'Newton',
    'Serangoon Gardens': 'Serangoon North',
    'Upper Bukit Timah, Clementi Park, Ulu Pandan': 'Ulu Pandan',
    'Hillview': 'Bukit Panjang',
    'Changi Business Park': 'Changi',
    'Kaki Bukit': 'Chai Chee',
    'one-north': 'Buona Vista',
    'Kallang Way': 'Whampoa',
    'Sungei Kadut': 'Kranji Reservoir',
    'Singapore River': 'Dhoby Ghaut',
    'Outram': 'Tanjong Pagar',
    'Potong Pasir': 'Toa Payoh',
    'Western Islands': 'Jurong Island',
    'Western Water Catchment': 'Tengah',
    'Central Water Catchment': 'Macritchie Reservoir',
    'Katong': 'Marine Parade',
    'Pioneer': 'Jurong (West)',
    'Orchard': 'Somerset (Road)',
    'River Valley': 'Somerset (Road)',
    'Alexandra': 'Queenstown',
    'Museum': 'Dhoby Ghaut'
})

In [17]:
cases_df3['suburb'].value_counts()

Chai Chee                  7788
Buangkok                   6121
Ang Mo Kio                 5054
Tai Seng                   4902
Serangoon                  4167
Admiralty                  3340
Simei                      2943
Bukit Panjang              2556
Jurong (West)              2395
Yishun                     2311
Whampoa                    2239
Choa Chu Kang (Central)    2202
Toa Payoh                  1964
Pasir Ris (Central)        1562
Marine Parade              1369
Newton                      980
Sembawang                   761
Dhoby Ghaut                 683
Jurong (East)               682
Kampong Bahru               404
Clementi                    300
Buona Vista                 295
Punggol                     289
Queenstown                  266
Ulu Pandan                  246
Botanic Garden              218
Changi                      195
Somerset (Road)             193
Serangoon North             124
Bukit Timah                 111
Tanjong Pagar                90
Macritch

In [18]:
cases_df3['suburb'].nunique()

41

In [19]:
cases_df3.head()

,no_of_cases,address,latitude,longitude,cluster_no,recent_cases,total_cases,date,month,postal_code,suburb
0,4,bishan street 22 (block 232),1.358286,103.845226,1,24,83,2015-07-03,7,570282,Ang Mo Kio
1,7,bishan street 22 (block 233),1.358639,103.845259,1,24,83,2015-07-03,7,570233,Ang Mo Kio
2,3,bishan street 22 (block 234),1.358390,103.845955,1,24,83,2015-07-03,7,570233,Ang Mo Kio
3,1,bishan street 22 (block 235),1.358719,103.846477,1,24,83,2015-07-03,7,570227,Ang Mo Kio
4,4,bishan street 22 (block 236),1.359041,103.846849,1,24,83,2015-07-03,7,570227,Ang Mo Kio


In [20]:
# Drop unncessary columns
cases_df3.drop(['cluster_no','month'],axis=1,inplace=True)
cases_df3.rename({'suburb':'region'},axis=1,inplace=True)
cases_df3.head()

,no_of_cases,address,latitude,longitude,recent_cases,total_cases,date,postal_code,region
0,4,bishan street 22 (block 232),1.358286,103.845226,24,83,2015-07-03,570282,Ang Mo Kio
1,7,bishan street 22 (block 233),1.358639,103.845259,24,83,2015-07-03,570233,Ang Mo Kio
2,3,bishan street 22 (block 234),1.358390,103.845955,24,83,2015-07-03,570233,Ang Mo Kio
3,1,bishan street 22 (block 235),1.358719,103.846477,24,83,2015-07-03,570227,Ang Mo Kio
4,4,bishan street 22 (block 236),1.359041,103.846849,24,83,2015-07-03,570227,Ang Mo Kio


In [21]:
cases_df3['date'] = pd.to_datetime(cases_df3['date'])

In [22]:
cases_df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56976 entries, 0 to 56975
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   no_of_cases   56976 non-null  int64         
 1   address       56976 non-null  object        
 2   latitude      56976 non-null  float64       
 3   longitude     56976 non-null  float64       
 4   recent_cases  56976 non-null  int64         
 5   total_cases   56976 non-null  int64         
 6   date          56976 non-null  datetime64[ns]
 7   postal_code   56976 non-null  int64         
 8   region        56976 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(4), object(2)
memory usage: 3.9+ MB


In [23]:
cases_df3.to_csv('../Data/dengue_cases_region.csv', index = False)

#### Data Dictionary

|Feature|Type|Dataset|Description|
|:---|:---|:---|:---|
|no_of_cases|integer|dengue cluster (NEA)|Number of reported dengue cases at this location| 
|address|object|dengue cluster (NEA)|Street address where dengue cases are reported (down to the apartment block level)| 
|latitude|float|dengue cluster (NEA)|Latitude of the street address| 
|longitude|float|dengue cluster (NEA)|Longitude of the street address| 
|recent_cases|integer|dengue cluster (NEA)|Number of dengue cases with onset in the last 2 weeks| 
|total_cases|integer|dengue cluster (NEA)|Tdengue cluster (NEA)|otal number of dengue cases reported in this cluster| 
|date|datetime|dengue cluster (NEA)|Date in YYYY-MM-DD format| 
|postal_code|integer|dengue cluster (NEA)|Postal code of the stress address| 
|region|object|dengue cluster (NEA) & postal mapping (URA)|Region of the street address| 